## 🤖 Uso de IA en el proyecto

Durante el desarrollo de este proyecto se usó un asistente de IA como apoyo para estructurar el flujo de trabajo, generar ejemplos de código y mejorar las explicaciones.  

Algunos de los **prompts utilizados** fueron:

1. *"Actúa como un científico de datos. Mi DataFrame de pandas podría tener valores nulos. ¿Cuáles son las estrategias más comunes para manejarlos? Dame el código para verificar cuántos nulos hay por columna y luego para eliminarlos."*  

2. *"Explícame cómo interpretar el resultado de `df.info()` en pandas y qué conclusiones se pueden sacar sobre la calidad de los datos."*  

3. *"Dame ejemplos de gráficos en Seaborn y Matplotlib para visualizar la distribución de variables y detectar outliers."*  

4. *"Enséñame cómo crear una aplicación básica en Streamlit con un menú lateral y gráficos interactivos."*  

Estos prompts fueron usados como **guía inicial**. Posteriormente, el código fue adaptado, corregido y documentado para ajustarse al proyecto de Ingeniería Agrícola.  



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Conexión con Google Drive
Montamos Google Drive para acceder a la carpeta del proyecto `proyecto_agricola/`.  
Esto nos permite trabajar de forma organizada, guardando tanto el dataset como el notebook en la misma estructura de carpetas.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

### Librerías utilizadas
- **pandas:** manejo de datos en formato tabla (DataFrames).  
- **seaborn y matplotlib:** visualización de datos estáticos.  
- **plotly:** visualización interactiva en 3D.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Proyectó Agrícola /Crop_recommendation.csv")
df.head()

### Carga del dataset
Se cargó el archivo `Crop_recommendation.csv` desde la carpeta `data/`.  
El comando `df.head()` muestra las **primeras 5 filas** del dataset, lo que nos permite verificar:  
- Que las columnas tienen nombres correctos (`N`, `P`, `K`, `temperature`, `humidity`, `ph`, `rainfall`, `label`).  
- Que los valores son coherentes (ej: pH entre 5 y 8, lluvia alrededor de 100–200 mm).

In [ ]:
df.info ()

### Análisis de `df.info()`
- El dataset contiene **2200 registros (filas)** y **8 columnas**.  
- Todas las columnas tienen `2200 non-null`, por lo tanto **no existen valores faltantes**.  
- **Tipos de datos:**  
  - `int64`: nutrientes N, P, K.  
  - `float64`: temperatura, humedad, pH, lluvia.  
  - `object`: cultivo (`label`).  
- El dataset es **ligero (~137 KB)** y no requiere tratamiento de nulos.

In [ ]:
df.describe()

### Estadísticas descriptivas (`df.describe()`)
- **Promedio:** Temperatura ~25.6 °C, humedad ~71%, pH ~6.4 (ligeramente ácido).  
- **Mínimos y máximos:**  
  - Temperatura: 8.8 °C – 43.6 °C.  
  - pH: 3.5 – 9.9 (hay algunos valores extremos).  
  - Lluvia: 20 – 298 mm.  
- **Percentiles:** permiten analizar la distribución de cada variable.  
Con esto confirmamos que los valores son **realistas**, aunque con algunos extremos en pH y lluvia.

In [ ]:
df['label'].value_counts()

### Distribución de cultivos (`label`)
Este comando cuenta cuántas veces aparece cada cultivo en el dataset.  
Se confirma que todos los cultivos tienen un número equilibrado de registros (~100 cada uno).  
Esto es importante porque evita **sesgos en el análisis**.

In [ ]:
df.describe(include='object')

### Estadísticas de variables categóricas
Aquí vemos:  
- Número de categorías únicas (`unique`).  
- Categoría más frecuente (`top`).  
- Frecuencia de la categoría más común (`freq`).  

En `label`, hay 22 cultivos distintos y ninguno domina excesivamente el dataset.

In [ ]:
print("Número de filas:", df.shape[0])
print("Número de columnas:", df.shape[1])

### Dimensiones del dataset
- **Filas:** 2200 registros.  
- **Columnas:** 8 variables.  
Esto confirma el tamaño total de la base de datos.

In [ ]:
df.sample(10)

### Muestra aleatoria de registros
`df.sample(10)` muestra 10 filas seleccionadas al azar.  
Esto ayuda a verificar la coherencia de los datos en diferentes partes del dataset.

In [ ]:
bins = [0, 60, 120, float('inf')]
labels = ['Baja', 'Media', 'Alta']
df['rainfall_level'] = pd.cut(df['rainfall'], bins=bins, labels=labels, right=False)
df[['rainfall', 'rainfall_level']].head(10)

### Feature Engineering: niveles de lluvia
Se creó una nueva variable categórica `rainfall_level` que clasifica la lluvia en:  
- **Baja:** <60 mm  
- **Media:** 60–120 mm  
- **Alta:** >120 mm  

Esto permite realizar análisis comparativos más fáciles, por ejemplo boxplots de temperatura según nivel de lluvia.

In [ ]:
sns.histplot(df['ph'], kde=True, bins=20, color="green")
plt.title("Distribución del pH del suelo")
plt.show()

In [ ]:
sns.boxplot(y=df['temperature'])
plt.title("Boxplot de la Temperatura")
plt.show()

In [ ]:
sns.regplot(x='K', y='P', data=df, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
plt.title("Relación entre K y P")
plt.show()

In [ ]:
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Mapa de correlación")
plt.show()

In [ ]:
fig = px.scatter_3d(df, x="N", y="P", z="K", color="label", title="Relación N-P-K por cultivo")
fig.show()

### Visualizaciones exploratorias
- **Histograma:** distribución del pH → la mayoría de los suelos están entre 5.5 y 7.5.  
- **Boxplot:** permite identificar outliers en temperatura y otras variables.  
- **Scatter:** muestra la relación entre nutrientes, con línea de tendencia.  
- **Mapa de calor:** revela correlaciones fuertes (ej: humedad y lluvia).  
- **Gráfico 3D (Plotly):** exploración interactiva de nutrientes N, P y K por cultivo.